# A Small Recap

Ideal Gas

Isentropic Process

* $pV^{\gamma}=k$; $p=k\rho^{\gamma}$ -> $dp=\gamma k \rho^{\gamma -1}d\rho$

Isentropic Compression

* $\frac{T_2}{T_1}=(\frac{p_2}{p_1})^{\frac{\gamma-1}{\gamma}}$  >>>or<<<   $\frac{p_2}{p_1}=(\frac{T_2}{T_1})^{\frac{\gamma}{\gamma-1}}$


Bernoulli / Compressible 

* $\frac{p_0}{p} = (1+ \frac{\gamma-1}{2} M^2)^{\frac{\gamma}{\gamma - 1}} = (1+ \frac{V^2}{2 c_p T_s})^{\frac{\gamma}{\gamma - 1}}$

Compression Efficiency

* $\eta_c = \frac{w_{c,s}}{w_c} = \frac{h_{2}^\prime-h_1}{h_2-h_1} = \frac{c_p(T_{2}^\prime-T_1)}{c_p(T_2-T_1)}$

![037](pictures/compression_efficiency.svg)


Some basic relationships

$M=\frac{V}{\sqrt{\gamma RT_{static}}}$

$\gamma R = c_p (\gamma - 1)$

# Inlet Diffuser

The conditions far away from the engine, at Station "a" (for ambient), are:

$T_{a_{static}}$, $P_{a_{static}}$, and $M_a$

If we now define *sub "0"* to mean "stagnation"...

so $T_{01}$ is the stagnation pressure at station 1;

   $T_{0a}$ is the stagnation pressure at station "a"

and $T_{a}$ is the static pressure at station "a"

>>*fluid properties parenthesis*

#### Isentropic flow up to duct inlet

Let's assume there are no losses from Station "a" to Station 1.

$P_{01} = P_{0a} = P_{a} (1 + \frac{\gamma - 1}{2}M_a^{2})^\frac{\gamma}{\gamma -1} = P_{a} (1+ \frac{V_{a}^2}{2 c_p T_a})^{\frac{\gamma}{\gamma - 1}}$

$T_{01} = T_{0a} = T_{a} (1 + \frac{\gamma - 1}{2}M_a^{2}) = T_{a}+ (\frac{V_{a}^2}{2 c_p} )$

#### Adiabatic Duct

Inside the inlet duct, we have friction losses, but no heat transfer

$T_{02}=T_{01}=T_{0a} $ -> stagnation temperatures are the same

$=> T_{02}=T_{01}= T_{1} + \frac{V_1^2}{2c_{p1}}$

$T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $

$=> T_{1} + \frac{V_1^2}{2c_{p1}} = T_{2} + \frac{V_2^2}{2c_{p2}} $

$T_{2} = T_{1} + \frac{V_1^2}{2c_{p1}} - \frac{V_2^2}{2c_{p2}}$

#### if it was isentropic...

And if we had isentropic compression, we could write:

$\frac{p_{02}}{p_{1}} = (\frac{T_{02}^\prime}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

which can be re-written as:

$\frac{p_{02}}{p_{1}} = (1 +  \frac{T_{02}^{'} - T_{1}}{T_{1}})^{\frac{\gamma}{\gamma -1}}$



#### considering the definition of inlet efficiency...



$\eta_i=\frac{T_{02}^{'}-T_{1}}{T_{02}-T_{1}}$

$T_{02}^{'}-T_{1}=\eta_i(T_{02}-T_{1})$ 

$\frac{p_{02}}{p_{1}} = (1 +  \frac{\eta_i(T_{02}-T_{1})}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

But $T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $


$\frac{p_{02}}{p_{1}} = (1 + \frac{\eta_i V_1^2}{2c_{p1}T_{1}})^{\frac{\gamma}{\gamma -1}}$

#### mass flow

$\dot{m}=\rho V A$

In [ ]:
#01 - preamble, imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [12, 7]
import cantera as ct
import ISA_module as ISA


In [ ]:
ct.__version__

In [ ]:
def get_p(ps:float, 
          gamma:float, 
          M:float)->float:
    '''
    This function calculates the stagnation pressure for isentropic process, given:
    
    ps: static pressure
    gamma: cp/cv for the gas
    M: Mach number
    '''
    return ps * ((1 + ((gamma - 1) / 2) * M**2)**(gamma / (gamma - 1)))


def get_T(Ts:float, 
          gamma:float, 
          M:float)->float:
    '''
    This function calculates the stagnation temperature for isentropic process, given:
    
    Ts: static temperature
    gamma: cp/cv for the gas
    M: Mach number
    '''
    return Ts * (1 + ((gamma - 1) / 2) * M**2)

In [ ]:
def get_Ts(T:float, 
           gamma:float, 
           M:float)->float:
    '''
    This function calculates the static temperature for isentropic process, given:
    
    T: stagnation or toal temperature
    gamma: cp/cv for the gas
    M: Mach number
    '''
    return T / (1 + ((gamma - 1) / 2) * M**2)



def get_ps(p:float, 
           Ts:float, 
           Tt:float, 
           gamma:float)->float:
    '''
    This function calculates the static pressure for isentropic process, given:
    
    p: stagnation pressure
    Ts: static temperature
    Tt: stagnation temperature
    gamma: cp/cv for the gas
    '''
    return p * (Ts / Tt)**(gamma / (gamma - 1))

In [ ]:
def get_gamma(gas:ct.Solution)->float:
    '''
    This function calculates gamma, given:
    
    gas: Cantera Solution object
    '''
    return gas.cp / gas.cv



def get_R(gas:ct.Solution)->float:
    '''
    This function calculates the gas constant, given:
    
    gas: Cantera Solution object
    '''
    return gas.cp - gas.cv



def get_a(gas:ct.Solution)->float:
    '''
    This function calculates the local speed of sound, given:
    
    gas: Cantera Solution object
    '''
    return np.sqrt(get_R(gas) * get_gamma(gas) * gas.T)

In [ ]:
def print_prop(gas:ct.Solution):
    '''
    This function prints the properties for a given station
    
    gas : Cantera object with gas conditions
    '''
    
    print(f'p = {(gas.P / ct.one_atm):2.2f} atm,',
      f'T = {(gas.T):2.0f} K,',
      f'h = {(gas.enthalpy_mass):2.0f} kJ/kg,',
      f's = {(gas.entropy_mass):2.0f} kJ/kg,')

    
def print_stations(st:list, station_names:dict, gas:dict, M:dict):
    '''
    This function prints the properties of all stations
    
    st : list with stations
    station_names : dictionary with station names
    gas : dictionary with gas objects
    M : dictionary with Mach numbers
    '''
    
    for station in st:
        print(f'station {station:<7} ({station_names[station]:<22}): Mach {M[station]:.3f}, ', end="")
        print_prop(gas[station])